# Data Acquisition & Import

## Sharing our Data for Replicable Science
The primary data source in this project was the **Million Playlist Dataset** provided by Spotify as part of the RecSys Challenge 2018.
https://labs.spotify.com/2018/05/30/introducing-the-million-playlist-dataset-and-recsys-challenge-2018/
The challenge ended on June 30th of 2018, so we used an archival copy of the dataset provided to us by the course staff.
All source code for this project is on a public GitHub Respository 
https://github.com/IACS-CS-209-Group44/Spotify

Unfortunately, Git and GitHub are not well suited to working on large files, especially large binary files.
Because of the large size of this data set, we were forced to use a shared folder on Dropbox to share large files.
This makes it more challenging for us to share our data with the public and create a conveniently reproducible set of calculations.  We are committed to the goal of fully reproducible science including data science with large files.
For a full scale research undertaking with a suitable budget, two ideas to better achieve this goal would be to host a publicly available database instance using a service like AWS and / or to share a container instance.  These techniques are beyond the scope of this project.  

The data on our Dropbox includes several formats that make it easy to replicate almost all of our work.  The h5 data files allow all of our imported data and results of calculations into Pandas dataframes.  There is also a database backup available in the file `SpotifyDB.bak`.  Warning: this is a bit of a monster, weighing in at **165 GB**, so it's not for the faint of heart.  Someone with access to a moderately powerful SQL Server instance, either on-premises or on the cloud, can restore this database from the backup file.
Anyone who is interested in accessing our dataset may email me at mse99@g.harvard.edu or michael.s.emanuel@gmail.com to request access to this dataset.  Any reasonable request will be granted read only access to the shared folder on Dropbox for as long as it is up.

## The JSON Data Set and the Choice of SQL Server Backend
The data provided by the mpd consists of 1000 json files.  Each file is a "slice" of 1,000 playlists.  There are 1,000 of these slice files that together comprise the 1,000,000 playlists in the data set.  The JSON files are a highly denormalized representation of the data, meaning they have a large amount of duplicated or redundant data.  As one prominent example, each spotify track has a unique identifier, and a track name associated with this identifier.  The JSON files duplicate the full track name when describing each track entry.  We stored the data in a fully normalized format, with separate database tables for the logical entities of a Track, Playlist, PlaylistEntry, etc.  I will describe our data model in greater detail below.  

The choice to use SQL Server as our back end for data was an important strategic decision on this project.  
The JSON respresentation of the data is convenient for a person reviewing the data, but is extremely inefficient for doing large scale computations.  As one example of how slow the JSON / Python API is, the data set comes with some Python utilities that do very simple summary calculations such as tabulating the most popular tracks.  This utility was on pace to take over 90 minutes to run on my desktop PC, which has top of the line hardware for a desktop computer.

It was clear that we needed a far more efficient back end for storing our data in a more normalized form.  Most people in this course would probably have chosen to use Pandas and persisted a series of data frames.  That is a completely sound choice.  In my particular case, I have spent many years working with SQL (first MySQL, then SQL Server).  I have also invested a large amount of time and money configuring an instance of SQL Server running on a high performance server sitting on a rack in my basement.  By comparison, I am very new to Pandas.  I'm now proficient for basic tasks, but I have often spent hours trying to figure out how to do an operation I could do in a matter of minutes in SQL Server.  It is often true that the best tool for a given job is the one you know how to use.  For me and this problem, that tool was SQL Server.

## Data Import into SQL Server
The first step was to understand the data and break it up into logical entities.  Each logical entity corresponds to its own database table.  The four most important logical entities in this data set are Artist, Album, Track, and Playlist.  I am going to go out of order for a moment and jump ahead to the Pandas dataframes that we used for downstream computations.  The code below loads these frames from an h5 data file.  

Before this code will run, please copy the files `data.h5` and `playlist_entry.h5` from the Dropbox folder to the directory where you cloned the GitHub repo.  On my system, the repo is cloned to `D:\IACS-CS-209-Spotify\` and the Dropbox folder is at `D:\Dropbox\IACS-CS-209-Spotify\`.  I copy the file `D:\Dropbox\IACS-CS-209-Spotify\mpd\database_export\h5` into `D:\IACS-CS-209-Spotify\data\data.h5`.  I do the analogous operation for files `playlist_entry.h5` and `track_pair.h5`.

It is possible to automate all of this in a very slick and convenient way using GitHub LFS.  https://git-lfs.github.com/
But this is a paid service that costs at least \$10 a month, possibly quite a bit more given the size of this data set.  I spent several hours investigating alternatives that would allow us to bypass this manual copy step to synchronize the Dropbox folder, but did not come up with a better procedure.  Once the file `data.h5` is in place, the code below will load the data frames for Artist, Album, Track, and Playlist into memory.

In [1]:
import pandas as pd
import time
from IPython.display import display
from typing import List, Dict, Optional

In [2]:
def load_frames(frame_names: Optional[List[str]] = None) -> Dict[str, pd.DataFrame]:
    """Load all available data frames.  Return a dictionary keyed by frame_name."""
    # Relative path to h5 data files
    path_h5 = '../data/'
    
    # Dictionary of dataframes to be generated.
    # Key = frame name, value = fname_h5
    frame_tbl: Dict[str, str] = {
        # Basic schema
        'Artist': 'data.h5',
        'Album': 'data.h5',
        'Track': 'data.h5',
        'Playlist': 'data.h5',
    
        # Tables relating to prediction outcomes and scoring
        'TrainTestSplit': 'data.h5',
        'Playlist_Last10': 'data.h5',
        'Playlist_trn': 'data.h5',
        'Playlist_tst': 'data.h5',
        
        # Tables relating to the baseline and playlist name prediction models
        'TrackRank': 'data.h5',
        'PlaylistName': 'data.h5',
        'PlaylistSimpleName': 'data.h5',
        'TrackRankBySimpleName': 'data.h5',
    
        # PlaylistEntry table is big - saved int its own file
        'PlaylistEntry': 'playlist_entry.h5',
    
        # Audio features
        'AudioFeatures': 'data.h5',
        'Genre': 'data.h5',
        'MetaGenre': 'data.h5',
        'TrackGenre': 'data.h5',
        'TrackMetaGenre': 'data.h5',
        
        # TrackPairs table is big - saved in its own file
        'TrackPairs': 'track_pairs.h5',
        
        # Scores of three models: baseline, playlist name, naive bayes
        'Scores_Baseline': 'data.h5',
        'Scores_SimpleName': 'data.h5',
        'Scores_TrackPair': 'data.h5',
        'Scores_Stack': 'data.h5',
        
        # Survey responses
        'SurveyResponse': 'data.h5',
        'SurveyPlaylist': 'data.h5',
        'SurveyPlaylistEntry': 'data.h5',

        # Artists being promoted by policy (mid-tier, female)
        'PromotedArtist': 'data.h5',

        # Survey recommendations
        'SurveyRecommendations': 'data.h5',
        'SurveyRecommendationsPromoted': 'data.h5',
        }
    
    # Set frame_names to all tables if it was not specified
    if frame_names is None:
        frame_names = frame_tbl.keys()
    
    # Start timer
    t0 = time.time()
    # Dictionary of data frames
    frames: Dict[str, pd.DataFrame] = dict()
    # Iterate over entries in frame_names, loading them from h5 files
    for frame_name in frame_names:
        # h5 filename for this frame
        fname_h5 = frame_tbl[frame_name]
        # Read the data frame
        frames[frame_name] = pd.read_hdf(path_h5 + fname_h5, frame_name)
        # Status update
        print(f'Loaded {frame_name}.')
    
    # Status update
    t1 = time.time()
    elapsed = t1 - t0
    print(f'\nLoaded {len(frames)} Data Frames.')
    print(f'Elapsed Time: {elapsed:0.2f} seconds.')
    return frames

**Load the frames for Artist, Album, Track, Playlist & PlaylistEntry into memory**

In [9]:
frames = load_frames(['Artist', 'Album', 'Track', 'Playlist', 'PlaylistEntry'])

Loaded Artist.
Loaded Album.
Loaded Track.
Loaded Playlist.
Loaded PlaylistEntry.

Loaded 5 Data Frames.
Elapsed Time: 4.72 seconds.


Notice how much faster and more efficient this is than loading the JSON files.  An analogous operation run directly on the JSON files took on the order of multiple minutes on my system.  The size of the decompressed JSON files was approximately 32 GB.  The size of the normalized tables storing the exact same information was approximately 2 GB.  Normalizing the data compressed it by a factor of 16.  (Try doing that with a ZIP file compression!)

It is said that a picture is worth a thousand words.  The same is often true of an example.  Please see below examples of the first 10 records on the five key tables in the data set: Artist, Album, Track, Playlist, and PlaylistEntry.

## Data Model for Artist, Album, Track, Playlist  & PlaylistEntry

In [4]:
display(frames['Artist'].head(10))

,ArtistID,ArtistUri,ArtistName
0,1,spotify:artist:0001cekkfdEBoMlwVQvpLg,Jordan Colle
1,2,spotify:artist:0001wHqxbF2YYRQxGdbyER,Motion Drive
2,3,spotify:artist:0001ZVMPt41Vwzt1zsmuzp,Thyro & Yumi
3,4,spotify:artist:0004C5XZIKZyd2RWvP4sOq,"""Faron Young, Nat Stuckey"""
4,5,spotify:artist:000DnGPNOsxvqb2YEHBePR,The Ruins
5,6,spotify:artist:000Dq0VqTZpxOP6jQMscVL,Thug Brothers
6,7,spotify:artist:000h2XLY65iWC9u5zgcL1M,Kosmose
7,8,spotify:artist:000spuc3oKgwYmfg5IE26s,Parliament Syndicate
8,9,spotify:artist:000UUAlAdQqkTD9sfoyQGf,Darren Gibson
9,10,spotify:artist:000UxvYLQuybj6iVRRCAw1,Primera Etica


In [5]:
display(frames['Album'].head(10))

,AlbumID,AlbumUri,AlbumName
0,1,spotify:album:00010fh2pSk7f1mGIhgorB,Okkadu (Original Motion Picture Soundtrack)
1,2,spotify:album:00045VFusrXwCSietfmspc,Let Love Begin Remixed
2,3,spotify:album:0005lpYtyKk9B3e0mWjdem,Stability
3,4,spotify:album:0005rH90S3le891y5XzPg4,"Mozart: Piano Concerto No. 27, KV595"
4,5,spotify:album:0008WZMLnvEBVnq418uZsI,Smart Flesh
5,6,spotify:album:0009lq7uJ6cW3Cxtf8eNUp,Earth: The Pale Blue Dot (Instrumental)
6,7,spotify:album:000aG92zPFtZ0FRLaaJHE5,X
7,8,spotify:album:000f3dTtvpazVzv35NuZmn,"Make It Fast, Make It Slow"
8,9,spotify:album:000g9ysmwb8NNsd4u1o087,"Nennt es, wie Ihr wollt"
9,10,spotify:album:000gdWY9uR4VYS5oZudY5o,Pérez Prado. Sus 40 Grandes Canciones


In [16]:
display(frames['Track'].head(10))

,TrackID,ArtistID,AlbumID,TrackUri,TrackName
0,1,208716,355266,spotify:track:0000uJA4xCdxThagdLkkLR,Heart As Cold As Stone
1,2,110598,6666,spotify:track:0002yNGLtYSYtc0X6ZnFvp,Muskrat Ramble
2,3,93681,77525,spotify:track:00039MgrmLoIzSpuYKurn9,Thas What I Do
3,4,5377,586855,spotify:track:0003Z98F6hUq7XxqSRM87H,???? ?????? ??? ???
4,5,285766,426742,spotify:track:0004ExljAge0P5XWn1LXmW,Gita
5,6,240510,337666,spotify:track:0005rgjsSeVLp1cze57jIN,Mi Razón de Ser
6,7,111429,625930,spotify:track:0005w1bMJ7QAMl6DY98oxa,"Sonata in G Major, BuxWV 271: Allegro -"
7,8,102394,113691,spotify:track:0006Rv1e2Xfh6QooyKJqKS,Nightwood
8,9,240180,287678,spotify:track:0007AYhg2UQbEm88mxu7js,Mandarin Oranges Part 2
9,10,35528,570255,spotify:track:0009mEWM7HILVo4VZYtqwc,Movement


In [17]:
display(frames['Playlist'].head(10))

,PlaylistID,PlaylistName,NumTracks,NumArtists,NumFollowers,NumEdits,DurationMS,IsCollaborative,ModifiedAt
0,0,Throwbacks,52,37,1,6,11532414,0,1493424000
1,1,Awesome Playlist,39,21,1,5,11656470,0,1506556800
2,2,korean,64,31,1,18,14039958,0,1505692800
3,3,mat,126,86,1,4,28926058,0,1501027200
4,4,90s,17,16,2,7,4335282,0,1401667200
5,5,Wedding,80,56,1,3,19156557,0,1430956800
6,6,I Put A Spell On You,16,13,1,2,3408479,0,1477094400
7,7,2017,53,48,1,38,12674796,0,1509321600
8,8,BOP,46,23,2,21,9948921,0,1508976000
9,9,old country,21,18,1,10,4297488,0,1501804800


In [18]:
display(frames['PlaylistEntry'].head(10))

,PlaylistID,Position,TrackID
0,0,0,236619
1,0,1,1866537
2,0,2,260403
3,0,3,347127
4,0,4,451364
5,0,5,270971
6,0,6,1784688
7,0,7,938244
8,0,8,2145897
9,0,9,776743


### Comments on the Table Design
The design of these database tables follows a few simple best practices.  All tables have an integer ID as their primary key.  This gives a large boost to the performance of queries that join tables.  Searching for an integer entry in an index is a much faster operation than comparings strings.  The original JSON data model did not have any integer IDs for any of these entities besides for the playlist ID.  All entities that exist in Spotify also have a field that Spotify names a Uri.  These are string identefiers.  These fields are equipped with unique constraints, building in both a data integrity check and causing SQL server to build indexes that both enforce the constraint and support fast joins on these fields.

The Track table demonstrates foreign key relationships.  The fields ArtistID and AlbumID are foreign keys onto the Artist and Album tables, respectively.  Note that no redundant information such as the ArtistName or ArtistUri are stored in the Track table.  Any consumer of this information is expected to get it by joining the Artist table using ArtistID.  The foreign key relationships are enforeced with foreign key constraints.  The primary key on the Track table is TrackID, and there is a separate unique constraint on the TrackUri.

This table also demonstrates a consistent naming scheme followed in the SpotifyDB database.  The name of the integer primary key on the table Artist is ArtistID.  This is one popular approach.  Another approach is to name the field ID.  I prefer to name the field ArtistID because then when you join from the Track table to the Artist table, the join clause uses the field AristID on both sides of the equality.  There are multiple naming approaches that are strong.  As is often the case, the most important thing is to pick one strategy and then follow it consistently.

For all you SQL aficianados out there, below please find the SQL table definitions for the main logical entities comprising the MPD dataset.  (Don't worry, I won't include all 42 tables in the database, just a few of the important ones!).  The entirety of the SQL used in this project can be found in the `sql` directory under the GitHub repo.  The files are named with a numeric prefix, so that tables are built in the correct order.  As an example, the Track table references the Artist and Album tables, so Artist and Album must be created first.  The relevant SQL scripts are named `03_MakeTable_Artist.sql`,  `04_MakeTable_Album.sql`, and `05_MakeTable_Track.sql`.

### SQL Table Definitions for Main Logical Entities

**Artist** / Script `03_MakeTable_Artist.sql`

```mysql
DROP sequence IF EXISTS dbo.SEQ_ArtistID
CREATE sequence dbo.SEQ_ArtistID
  AS INT start WITH 1 increment BY 1 NO cycle;
  
DROP TABLE IF EXISTS dbo.Artist;
CREATE TABLE dbo.Artist(
ArtistID INT NOT NULL
  DEFAULT next value FOR dbo.SEQ_ArtistID,
ArtistUri CHAR(37) NOT NULL,
ArtistName VARCHAR(512) NOT NULL,
-- Primary Key and Unique constraints
CONSTRAINT PK_Artist_ArtistID PRIMARY KEY (ArtistID),
CONSTRAINT UNQ_Artist_ArtistUri UNIQUE(ArtistUri),
-- ArtistName should be unique, but unfortunately it's not; index it instead
INDEX IDX_Artist_ArtistName (ArtistName),
);
```

**Album** / Script `04_MakeTable_Album.sql`

```mysql
DROP sequence IF EXISTS dbo.SEQ_AlbumID
CREATE sequence dbo.SEQ_AlbumID
  AS INT start WITH 1 increment BY 1 NO cycle;
  
DROP TABLE IF EXISTS dbo.Album;
CREATE TABLE dbo.Album(
AlbumID INT NOT NULL
  DEFAULT next value FOR dbo.SEQ_AlbumID,
AlbumUri CHAR(36) NOT NULL,
AlbumName VARCHAR(512) NOT NULL,
-- Primary Key and Unique constraints
CONSTRAINT PK_Album_AlbumID PRIMARY KEY (AlbumID),
CONSTRAINT UNQ_Album_AlbumUri UNIQUE(AlbumUri),
-- AlbumName should be unique, but unfortunately it's not; index it instead
INDEX IDX_Album_AlbumName (AlbumName),
);
```

**Track** / Script `05_MakeTable_Track.sql`

```mysql
DROP sequence IF EXISTS dbo.SEQ_TrackID
CREATE sequence dbo.SEQ_TrackID
  AS INT start WITH 1 increment BY 1 NO cycle;
  
DROP TABLE IF EXISTS dbo.Track;
CREATE TABLE dbo.Track(
TrackID INT NOT NULL
  DEFAULT next value FOR dbo.SEQ_TrackID,
ArtistID INT NOT NULL,
AlbumID INT NOT NULL,
TrackUri CHAR(36) NOT NULL,
TrackName VARCHAR(512) NOT NULL,
-- Primary Key and Unique constraints
CONSTRAINT PK_Track_TrackID PRIMARY KEY (TrackID),
CONSTRAINT UNQ_Track_TrackUri UNIQUE(TrackUri),
-- TrackName should be unique, but unfortunately it's not; index it instead
INDEX IDX_Track_TrackName (TrackName),
-- Foreign keys on ArtistID and AlbumID
CONSTRAINT FK_Track_ArtistID FOREIGN KEY (ArtistID) 
  REFERENCES dbo.Artist(ArtistID),
CONSTRAINT FK_Track_AlbumID FOREIGN KEY (AlbumID) 
  REFERENCES dbo.Album(AlbumID)
);
```

**Playlist** / Script `06_MakeTable_Playlist.sql`

```mysql
CREATE TABLE dbo.Playlist(
PlaylistID INT NOT NULL,
PlaylistName varchar(100) NOT NULL,
-- Playlist attributes
NumTracks SMALLINT NOT NULL,
NumArtists SMALLINT NOT NULL,
NumFollowers INT NOT NULL,
NumEdits SMALLINT NOT NULL,
DurationMS INT NOT NULL,
IsCollaborative BIT NOT NULL,
ModifiedAt INT NOT NULL,
-- Primary key
CONSTRAINT PK_Playlist_PlaylistID PRIMARY KEY (PlaylistID),
-- Indices
INDEX IDX_Playlist_PlaylistName (PlaylistName),
INDEX IDX_Playlist_NumFollowers(NumFollowers),
);
```

**PlaylistEntry** / Script `07_MakeTable_PlaylistEntry.sql`

```mysql
CREATE TABLE dbo.PlaylistEntry(
PlaylistID INT NOT NULL,
Position SMALLINT NOT NULL,
TrackID INT NOT NULL,
-- Primary key is the pair of the PlaylistID and Position
CONSTRAINT PK_PlaylistEntry PRIMARY KEY (PlaylistID, Position),
-- Foreign keys
CONSTRAINT FK_PlaylistEntry_PlaylistID
  FOREIGN KEY (PlaylistID) REFERENCES dbo.Playlist(PlaylistID),
CONSTRAINT FK_PlaylistEntry_TrackID
  FOREIGN KEY (TrackID) REFERENCES dbo.Track(TrackID),
);
```

## Importing the Raw Data into SQL Server

These empty tables are very nice and clean, but they don't look anything like the quite bloated JSON files.  How were they populated?  I used a two step process.  In the first step, I populated "raw" tables for Playlist and PlaylistEntry that followed the structure of the JSON file contents much more closely.  These are the definitions of the raw tables; they have the same table names as their sister tables in the main database, but are stored in a different schema `r` (for raw) rather than the gnomically named schema `dbo` which is the default schema in SQL Server, and short for DataBaseOwner. 

**Raw Table r.Playlist** / Script `01_MakeTable_r_Playlist`
```mysql
CREATE TABLE r.Playlist(
PlaylistID INT NOT NULL,
PlaylistName VARCHAR(1024) NOT NULL,
-- The number of tracks, albums, and artists on this playlist
NumTracks SMALLINT NOT NULL,
NumAlbums SMALLINT NOT NULL,
NumArtists SMALLINT NOT NULL,
-- Additional information about the playlist
NumFollowers INT NOT NULL,
NumEdits SMALLINT NOT NULL,
DurationMS INT NOT NULL,
IsCollaborative BIT NOT NULL,
ModifiedAt INT NOT NULL,
-- Primary Key and indices
CONSTRAINT PK_r_Playlist_PlaylistID PRIMARY KEY (PlaylistID),
INDEX IDX_r_Playlist_PlaylistName (PlaylistName)
)
```

**Raw Table r.PlaylistEntry** / Script `02_MakeTable_r_PlaylistEntry`
```
CREATE TABLE r.PlaylistEntry(
PlaylistID INT NOT NULL,
Position SMALLINT NOT NULL,
-- The track
TrackUri VARCHAR(256) NOT NULL,
TrackName VARCHAR(1024) NOT NULL,
-- The album
AlbumUri VARCHAR(256) NOT NULL,
AlbumName VARCHAR(1024) NOT NULL,
-- The artist
ArtistUri VARCHAR(256) NOT NULL,
ArtistName VARCHAR(1024) NOT NULL,
-- Additional track info
TrackDurationMS INT NOT null
-- Primary Key and indices
CONSTRAINT PK_r_PlaylistEntry PRIMARY KEY (PlaylistID, Position),
INDEX IDX_r_PlaylistEntry_Tracks (TrackUri, TrackName),
INDEX IDX_r_PlaylistEntry_Albums (AlbumUri, AlbumName),
INDEX idx_r_PlaylistEntry_Artists (ArtistUri, ArtistName)
)
```

The cells below present the contents of the file `src/db_import.py`.  This program was run from the terminal and it populated the two raw tables **r.Playlist** and **r.PlaylistEntry** by reading the JSON files.  In practice, if you are replicating our results the best strategy is probably to restore database from SpotifyDB.bak.  In principle, someone trying to replicate our work could run this script on their local system.  They would need to modify the function `getConnection()` though because it is currently configured to acquire a connection to my database instance, which is on a server named `Thor` and an instance named `Mjolnir`.  (Of course, if you are a fan of Norse mythology or the Marvel Universe, you might *already* have a SQL Server Instance with this name, but it seems... unlikely.)  Just change `Thor` to the name of your SQL Server and `Mjolnir` to the name of your instance and it should work with an on-premises setup.  I don't know the exact steps to configure it on AWS but it should be straightforward.

In [20]:
from sys import argv
import os
import json
import pyodbc
import time
from typing import List, Tuple, Dict


# *********************************************************************************************************************
# Mapping from strings to bool for IsCollaborative field
str2bool: Dict[str, bool] = {
        'true': True,
        'false': False,
        }


def get_insertions(fname: str) -> Tuple[List[Tuple], List[Tuple]]:
    """
    Read the mpd slice with this filename.
    Returns two lists of tuples, inserts_playlist and inserts_tracks.
    Each entry is a tuple matching one record on the r.Playlist and r.PlaylistEntry tables, respectively.
    """
    # Reference external variable used
    global str2bool
    # Open the file
    with open(fname) as fh:
        # Read the json contents
        js = fh.read()
    # Read in the data as a JSON object
    mpd_slice = json.loads(js)
    # Extract the playlists field from the slice
    playlists = mpd_slice['playlists']
    # Length (should be 1000)
    playlist_count: int = len(playlists)

    # Preallocate list of rows to be inserted for the Playlist table
    rows_playlist: List[Tuple] = playlist_count * [None]
    # Initialize an empty list of rows to be inserted for the PlaylistEntry table (we don't know its length yet)
    rows_playlist_entry: List[Tuple] = list()

    # Iterate over each playlist in the slice
    for i, playlist in enumerate(playlists):
        # Get the attributes of this playlist
        # Name attributes consistent with the database naming scheme
        # ID and name
        PlaylistID: int = playlist['pid']
        PlaylistName: str = playlist['name']
        # Number of tracks, albums, and artits
        NumTracks: int = playlist['num_tracks']
        NumAlbums: int = playlist['num_albums']
        NumArtists: int = playlist['num_artists']
        # Additional info
        NumFollowers: int = playlist['num_followers']
        NumEdits: int = playlist['num_edits']
        DurationMS: int = playlist['duration_ms']
        IsCollaborative: bool = str2bool[playlist['collaborative']]
        ModifiedAt: int = playlist['modified_at']

        # Assemble this into a tuple with one row to be inserted into r.Playlist
        row_playlist: Tuple = (PlaylistID, PlaylistName, NumTracks, NumAlbums, NumArtists,
                               NumFollowers, NumEdits, DurationMS, IsCollaborative, ModifiedAt)
        # Save this row to the inserts
        rows_playlist[i] = row_playlist

        # Get the tracks out of this playlist
        tracks: List[Dict] = playlist['tracks']
        # Iterate over the tracks
        for track in tracks:
            # Get the contents of this track - use database names and order
            # already have PlaylistID above
            Position: int = track['pos']
            # The track
            TrackUri: str = track['track_uri']
            TrackName: str = track['track_name']
            # The album
            AlbumUri: str = track['album_uri']
            AlbumName: str = track['album_name']
            # The artist
            ArtistUri: str = track['artist_uri']
            ArtistName: str = track['artist_name']
            # Duration
            TrackDurationMS: int = track['duration_ms']

            # Assemble this into a tuple with one row to be inserted into r.PlaylistEntry
            row_playlist_entry: Tuple = (PlaylistID, Position, TrackUri, TrackName, AlbumUri, AlbumName,
                                         ArtistUri, ArtistName, TrackDurationMS)
            rows_playlist_entry.append(row_playlist_entry)

    # Return the lists ready to be inserted into r.Playlist and r.PlaylistEntry
    return (rows_playlist, rows_playlist_entry)

In [23]:
# *********************************************************************************************************************
def getConnection() -> pyodbc.Connection:
    """Get database connection"""
    raise RuntimeError('Oops please do not run this again the table is already populated!')
    driver: str = r'{ODBC Driver 13 for SQL Server}'
    server: str = r'THOR\MJOLNIR'
    database: str = 'SpotifyDB'
    auth: str = 'Trusted_Connection=yes;'
    conn_string: str = f'DRIVER={driver};SERVER={server};DATABASE={database};{auth}'
    conn: pyodbc.Connection = pyodbc.connect(conn_string)
    return conn


# *********************************************************************************************************************
def delete_playlist(curs, PlaylistID_Min: int, PlaylistID_Max: int) -> None:
    """
    Deletes a block of rows in DB table r.Playlist
    INPUTS:
    ======
    curs:           Database cursor
    PlaylistID_min:   First PLaylistID to be deleted (inclusive)
    PlaylistID_max:   Last  PLaylistID to be deleted (exclusive)
    """

    # SQL string to delete records for this range of PlaylistID
    sqlDelete = '''
    DELETE FROM r.Playlist WHERE ? <= PlaylistID and PlaylistID < ?
    '''
    # Delete records in this range of PlaylistID
    curs.execute(sqlDelete, PlaylistID_Min, PlaylistID_Max)


def insert_playlist(curs, rows: List[Tuple]):
    """
    Inserts a list of rows into the DB table r.Playlist
    INPUTS:
    ======
    curs:           Database cursor
    rows_playlist:  Row of records to be inserted
    """

    # SQL string to insert ONE record into r.Playslist
    # row_playlist: Tuple = (PlaylistID, PlaylistName, NumTracks, NumAlbums, NumArtists,
    #                        NumFollowers, NumEdits, DurationMS, IsCollaborative, ModifiedAt)
    sqlInsert = '''
    INSERT INTO r.Playlist
    (PlaylistID, PlaylistName, NumTracks, NumAlbums, NumArtists,
    NumFollowers, NumEdits, DurationMS, IsCollaborative, ModifiedAt)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''
    # Insert batch of records using executemany method
    curs.executemany(sqlInsert, rows)
    # Commit changes
    curs.commit()


# *********************************************************************************************************************
def delete_playlist_entry(curs, PlaylistID_Min: int, PlaylistID_Max: int) -> None:
    """
    Deletes a block of rows in DB table r.PlaylistEntry
    INPUTS:
    ======
    curs:           Database cursor
    PlaylistID_min:   First PLaylistID to be deleted (inclusive)
    PlaylistID_max:   Last  PLaylistID to be deleted (exclusive)
    """

    # SQL string to delete records for this range of PlaylistID
    sqlDelete = '''
    DELETE FROM r.PlaylistEntry WHERE ? <= PlaylistID and PlaylistID < ?
    '''
    # Delete records in this range of PlaylistID
    curs.execute(sqlDelete, PlaylistID_Min, PlaylistID_Max)


def insert_playlist_entry(curs, rows: List[Tuple]):
    """
    Inserts a list of rows into the DB table r.PlaylistEntry
    INPUTS:
    ======
    curs:           Database cursor
    rows_playlist:  Row of records to be inserted
    """

    # SQL string to insert ONE record into r.PlaylistEntry
    # row_playlist_entry: Tuple = (PlaylistID, Position, TrackUri, TrackName, AlbumUri, AlbumName,
    #                              ArtistUri, ArtistName, TrackDurationMS)
    sqlInsert = '''
    INSERT INTO r.PlaylistEntry
    (PlaylistID, Position, TrackUri, TrackName, AlbumUri, AlbumName, ArtistUri, ArtistName, TrackDurationMS)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''
    # Insert batch of records using executemany method
    curs.executemany(sqlInsert, rows)
    # Commit changes
    curs.commit()


In [26]:
# *********************************************************************************************************************
def main():
    # Unpack arguments
    argc: int = len(argv)-1
    if argc == 0:
        sliceMin = 0
        sliceMax = 1000
    if argc == 1:
        sliceMin = 0
        sliceMax = int(argv[1])
    if argc == 2:
        sliceMin: int = int(argv[1])
        sliceMax: int = int(argv[2])
    if argc not in (0, 1, 2):
        print('Usage: python db_import.py sliceMin sliceMax.')
        print('This will insert slices from and including sliceMin, up to but not including sliceMax.')
        return
    # Range of PlaylistID's
    PlaylistID_Min: int = sliceMin * 1000
    PlaylistID_Max: int = sliceMax * 1000
    # Status update
    print(f'Beginning database import from slice {sliceMin} to {sliceMax}, '
          f'i.e. from PlaylistID {PlaylistID_Min} to {PlaylistID_Max}.')

    # Set the path of the MPD directory
    mpd_path = r'D:/Dropbox/IACS-CS-209-Spotify/mpd/data'
    # Move to this directory and get all filesnames; each file is a slice
    os.chdir(mpd_path)
    fnames: List[str] = os.listdir()
    # The size of each slice
    slice_size: int = 1000

    # Make a sorted list of mpd slice files
    fnames_mpd: List[Tuple[str, int]] = list()
    # Iterate over all the files in this directory
    for fname in fnames:
        # Filenames have the format e.g.  "mpd.slice.1000-1999.json"
        # First, check that this is an mpd data slice file; if not, skip it
        is_mpd_file: bool = fname.startswith("mpd.slice.") and fname.endswith(".json")
        if not is_mpd_file:
            continue
        # Extract the PlaylistID range from the file name
        pid_range: str = fname.split('.')[2]
        # The slice is the starting pid / 1000 (integer division)
        sliceNum: int = int(pid_range.split('-')[0]) // slice_size
        # If this slice is in the range, add (fname, sliceNum) to fnames_mpd
        if sliceMin <= sliceNum and sliceNum < sliceMax:
            fnames_mpd.append((fname, sliceNum))
    # Sort this by sliceNum
    fnames_mpd.sort(key=lambda x: x[1])

    # Get a database connection and a cursor; close the connection at the end!
    conn = getConnection()
    curs = conn.cursor()
    # Set mode to fast insertions on execute many
    curs.fast_executemany = True

    # Track progress
    num_processed: int = 0
    num_total: int = sliceMax - sliceMin
    # Start the timer
    t0 = time.time()

    for fname, sliceNum in fnames_mpd:
        # If we get here, this is a valid mpd data file in the range we want to process
        # Get the rows of data to insert into both tables
        rows_playlist, rows_playlist_entry = get_insertions(fname)
        # Range of PlaylistID in this slice
        PlaylistID_Min = min(pl[0] for pl in rows_playlist)
        PlaylistID_Max = max(pl[0] for pl in rows_playlist) + 1

        # Delete records in this range on the table r.Playlist
        delete_playlist(curs, PlaylistID_Min, PlaylistID_Max)
        # Insert records in this range on the table r.Playlist
        insert_playlist(curs, rows_playlist)

        # Delete records in this range on the table r.PlaylistEntry
        delete_playlist_entry(curs, PlaylistID_Min, PlaylistID_Max)
        # Insert records in this range on the table r.PlaylistEntry
        insert_playlist_entry(curs, rows_playlist_entry)

        # Status update
        num_processed += 1
        num_left = num_total - num_processed
        t1 = time.time()
        elapsed_time = t1 - t0
        average_pace = elapsed_time / num_processed
        projected_time = num_left * average_pace
        print(f'Processed slice {sliceNum} in {fname}.  '
              f'Elapsed time {round(elapsed_time)}, projected time {round(projected_time)} seconds.')

    # Close DB connection
    curs.close()
    conn.close()

## Populating the Normalized DB Tables from the Raw Tables

Now that the two raw tables r.Playlist and r.PlalyistEntry are populated, it's straightforward populate Artist, Album, Track, Playlist and PlaylistEntry in the dbo schema.  I refer to this operation as a 'data import.'  The scripts that do this are located in the repo folder `sql/03_DataImport`.  Here are the ones that generate the five principal tables

**Import Artist** / `01_ImportArtist.sql`
```mysql
INSERT INTO dbo.Artist
(ArtistName, ArtistUri)
SELECT
  pe.ArtistName,
  pe.ArtistUri
FROM
  r.PlaylistEntry AS pe
GROUP BY 
  pe.ArtistName, pe.ArtistUri
ORDER BY pe.ArtistUri;
```

**Import Album** / `02_ImportAlbum.sql`
```mysql
INSERT INTO dbo.Album
(AlbumName, AlbumUri)
SELECT
  pe.AlbumName,
  pe.AlbumUri
FROM
  r.PlaylistEntry AS pe
GROUP BY pe.AlbumName, pe.AlbumUri    
ORDER BY pe.AlbumUri;
```

**Import Track** / `03_ImportTrack.sql`
```mysql
INSERT INTO dbo.Track
(TrackName, TrackUri, ArtistID, AlbumID)
SELECT
  pe.TrackName,
  pe.TrackUri,
  ar.ArtistID,
  al.AlbumID
FROM
  r.PlaylistEntry AS pe
  INNER JOIN dbo.Artist AS ar ON
    ar.ArtistUri = pe.ArtistUri
  INNER JOIN dbo.Album AS al ON
    al.AlbumUri = pe.AlbumUri
GROUP BY pe.TrackName, pe.TrackUri, ar.ArtistID, al.AlbumID
ORDER BY pe.TrackUri;
```

**Import Playlist** / `03_ImportPlaylist.sql`
```mysql
INSERT INTO dbo.Playlist
(PlaylistID, PlaylistName, NumTracks, NumArtists, NumFollowers, NumEdits,
 DurationMS, IsCollaborative, ModifiedAt)
SELECT
  pl.PlaylistID,
  TRIM(pl.PlaylistName) AS PlaylistName,
  pl.NumTracks,
  pl.NumArtists,
  pl.NumFollowers,
  pl.NumEdits,
  pl.DurationMS,
  pl.IsCollaborative,
  pl.ModifiedAt
FROM
  r.Playlist AS pl;
```

**Import PlaylistEntry** / `04_ImportPlaylistEntry.sql`
```mysql
INSERT INTO dbo.PlaylistEntry
(PlaylistID, Position, TrackID)
SELECT
  pe.PlaylistID,
  pe.Position,
  t.TrackID
FROM
  r.PlaylistEntry AS pe
  INNER JOIN dbo.Track AS t ON
    t.TrackUri = pe.TrackUri;
```

# Predicting the Last 10 Tracks of a Playlist

The Spotify app offers users a functionality suggesting tracks you might want to add to a playlist.  It recommends 10 tracks.  If you don't like any of them, it will recommend another 10, and so on.  We set as our first task building a predictive model that could predict the last 10 tracks of a playlist by looking at features including the first $n-10$ tracks and the name of the playlist.  This problem is easy to state but hard to solve in practice.  The data set is truly massive.  There are 1,000,000 playlist and 2.26 million distinct tracks appearing.  Scaling the problem down by considering only a smaller number of playlists at a time is a straightforward to make the scale more manageable.  But we found it very challenging to deal with the huge number of columns.

The first idea we discussed was to simply cut off consideration of tracks that were not among the most popular, e.g. the top 100,000 tracks.  There are a few issues with this.  This data set has a very long tail.  The first 100,000 tracks are dominant, but they only comprise 84\% of the total number of entries of the on the PlaylistEntry table.  (This calculation was done in the database view `v.TopTracks`).  And even if you did reduce the size of this data set to 100,000 columns, the scale is still so massive that dense matrices are infeasible.  With so many features, reducing the number of rows (playlists) you study is dangerous.  Suppose for the sake of discussion we reduced it by a factor of 10, dropping down to 100,000.  The matrix representing playlist entries would still have a size of 100,000 x 100,000, i.e. it would have $10^10$ or 10 billion entries in it.  That is a lot of memory and is not going to be feasible.

If all we wanted to do was one or two basic linear algebra operations, then it is feasible load this data into sparse matrices.  Care must be taken arrange the input into the right shape, and to only perform fast operations.  As an example, if you want to perform matrix multiplication on A * B, this will work efficiently if A is in 'csr' (compressed sparse row) and B is in 'csc' (compressed sparse column) format.  Otherwise it is going to be very slow.

The biggest problem with reducing the number of tracks under consideration though was the ambitious goal of the project: to produce recommendations that had a higher frequency of artists in a "promoted" group.  In this case, the promoted group consisted of female mid-tier artists.  Avriel generated a list of ~14,000 female midtier artists currently on Spotify, which matched up with about 10,000 artists in our dataset (out of about 295,000 artists total).  These artists accounted for a tiny 0.8576\% of total playlist recommendations with the default model which uses baseline frequencies.  The upshot is that a model that limits the tracks to the top 100,000 or even the top 200,000 is going to be hopeless at recommending tracks deep in this long tail-it may as well be guessing.

As we contemplated the extent of this challenge and the limited time available, we chose to concentrate on three very straightforward models:
* Baseline Model: predict tracks based solely on their overall frequency in the training data set
* Playlist Name Model: predict tracks based on one feature, the name of the playlist
* Naive Bayes (Track Pair) Model: Predict a playlist by analyzing the frequency with which pairs of tracks appear together in the same playlist.  Sum up the frequencies of the "other" tracks given the visible tracks in a playlist, and guess the top 10 other tracks
If time permitted, we also considered adding a fourth model:
* Stacked Model: Predict playlist using a combination of the playlist name and track pair information.

When I considered these three models, I realized that none of the first three required any "fancy" machine learning and could all be implemented cleanly in SQL.  Given the challenges of moving data from one platform to another and the limited time availabe, I chose to estimate the three basic models purely in SQL.  The benefit of doing this in SQL is that is highly scalable and running on powerful hardware.  All predictions were made on the full data set of one million playlists, split into 900,000 training and 100,000 test playlists.

Let's load in some additional dataframes so we can see examples of these predictions and their building blocks.

In [11]:
f1 = ['Artist', 'Album', 'Track', 'Playlist', 'PlaylistEntry']
f2 = ['TrackRank', 'PlaylistName', 'PlaylistSimpleName']
frames = load_frames(f1 + f2)

Loaded Artist.
Loaded Album.
Loaded Track.
Loaded Playlist.
Loaded PlaylistEntry.
Loaded TrackRank.
Loaded PlaylistName.
Loaded PlaylistSimpleName.

Loaded 8 Data Frames.
Elapsed Time: 3.88 seconds.


## The Baseline Model

This model is about as simple as you can get short of randomly guessing one of the 2.26 million tracks.  Here is how it works in words.  Assemble a list of the most popular tracks in descending order.  The frequency here is how often this track appears as an entry in the training data set.  To predict the last 10 tracks, take the list of candidates, remove the tracks that have already been played, and guess the top 10.  That's it!

Below are two cells showing the SQL table definition for the persisted predictions in this model, and the SQL script used to generate the predictions.  The exercise is conceptually trivial.  All of the action comes in the handling of details like monitoring the progress and designing the insert batch so it can be restarted midway.

`31_MakeTable_Prediction_Baseline.sql`

```mysql
CREATE TABLE dbo.Prediction_Baseline(
PlaylistID INT NOT NULL,
Position SMALLINT NOT NULL,
TrackID INT NOT NULL,
BaselineFrequency INT NOT NULL,
TrackRank INT NOT NULL,
CONSTRAINT PK_Prediction_Baseline_PlaylistID_Position
  PRIMARY KEY (PlaylistID, Position),
CONSTRAINT UNQ_Prediction_Baseline_PlaylistID_TrackID
  UNIQUE (PlaylistID, TrackID),
CONSTRAINT FK_Prediction_Baseline_PlaylistID
  FOREIGN KEY (PlaylistID) 
  REFERENCES dbo.Playlist(PlaylistID),
CONSTRAINT FK_Prediction_Baseline_TrackID
  FOREIGN KEY (TrackID)
  REFERENCES dbo.Track(TrackID),  
);
```

` 01_Predict_Baseline.sql`
```mysql
DECLARE @PlaylistCount AS INT = 1000000;
DECLARE @BatchSize AS INT = 1000;
DECLARE @i AS INT = 0;
DECLARE @p1 INT;
DECLARE @p2 INT;

WHILE (@i * @BatchSize) < @PlaylistCount
BEGIN

-- Range of playlists for this loop iteration
SET @p1 = (@i * @BatchSize + 1);
SET @p2 = ((@i+1) * @BatchSize);

-- Delete records in this block
DELETE pr
FROM dbo.Prediction_Baseline AS pr
WHERE pr.PlaylistID BETWEEN @p1 AND @p2;

-- CTE to get (PlaylistID, TrackID, BaselineFrequency)
WITH t1 AS (
SELECT
  pl.PlaylistID,
  pl.NumTracks,
  tr.TrackID,
  tr.Frequency AS BaselineFrequency,
  tr.TrackRank AS TrackRank
FROM
  -- Start with all playlists
  dbo.Playlist AS pl
  -- Track ranks up to 1024
  INNER JOIN dbo.TrackRank AS tr ON
    tr.TrackRank <= 1024
WHERE  
  -- Current block of PlaylistIDs
  pl.PlaylistID BETWEEN @p1 AND @p2
),
t2 AS (
SELECT
  t1.PlaylistID,
  row_number() OVER 
    (PARTITION BY t1.PlaylistID ORDER BY t1.BaselineFrequency DESC) 
    AS Position,
  t1.TrackID,
  t1.BaselineFrequency,
  t1.TrackRank
FROM
  t1
-- Don't pick tracks that are already on the first (n-10) elements of the playlist!
WHERE NOT EXISTS 
  (SELECT pe.PlaylistID FROM dbo.PlaylistEntry AS pe 
   WHERE 
    pe.PlaylistID = t1.PlaylistID AND 
    pe.TrackID = t1.TrackID AND 
    pe.Position <= t1.NumTracks - 11)
)
-- Insert the first 512 positions into the Prediction table
INSERT INTO dbo.Prediction_Baseline
(PlaylistID, Position, TrackID, BaselineFrequency, TrackRank)
SELECT
  t2.PlaylistID,
  t2.Position,
  t2.TrackID,
  t2.BaselineFrequency,
  t2.TrackRank
FROM
  t2
WHERE
  -- Only the first 512 positions
  t2.Position <= 512;

-- Status update; manual loop increment
PRINT CONCAT('Completed PlaylistID ', @i*@BatchSize+1, ' to ', (@i+1)*@BatchSize);
SET @i = @i+1;

END
```

**Top 50 playlists in our Data Set**

In [36]:
# Start with the top 50 tracks
t1 = frames['TrackRank'].sort_values(by=['TrackRank']).head(50)
# Join the Track table to get the TrackName ArtistID 
output_columns2 = ['TrackID', 'ArtistID', 'TrackName', 'TrackRank', 'Frequency']
t2 = pd.merge(left=t1, right=frames['Track'], on='TrackID', suffixes=('', '_dup'))
# Join the Artist table to get the ArtistName
output_columns3 = output_columns = ['TrackID', 'TrackName', 'ArtistName', 'TrackRank', 'Frequency']
t3 = pd.merge(left=t2, right=frames['Artist'], on='ArtistID', suffixes=('', '_dup'))[output_columns3]
display(t3.sort_values(by='TrackRank'))

,TrackID,TrackName,ArtistName,TrackRank,Frequency
0,2181619,HUMBLE.,Kendrick Lamar,1,41866
2,563659,One Dance,Drake,2,39143
6,2257296,Broccoli (feat. Lil Yachty),DRAM,3,37164
7,2093570,Closer,The Chainsmokers,4,37015
10,920373,Congratulations,Post Malone,5,35941
13,1572888,Caroline,Aminé,6,31602
14,670143,iSpy (feat. Lil Yachty),KYLE,7,31599
15,1309651,Bad and Boujee (feat. Lil Uzi Vert),Migos,8,31475
16,2144121,XO TOUR Llif3,Lil Uzi Vert,9,31430
17,315491,Location,Khalid,10,31094


## The Playlist Simple Name Model

This model is the second simplest model we could think of.  It only considers one feature of each playlist: the name of the playlist.  While this might sound simple, it is also a powerful feature.  The best way to see the power of this feature is to experiment with the Spotify app.  When I was putting together playlists so I could complete our survey, I started by typing in the names of three playlists.  I'm a classical music fan, and my playlists were titled "Virtuoso Piano", "Symphonies" and "Chamber Music".  Based only on these titles, Spotify gave me a roster of entirely plausible suggestions, tilted to be sure to the most popular works in each field, but all relevant.  The one million playlists included 73,498 distinct playlist names.  A review of these playlist names shows there are a lot of effective duplicates separated only by letter casing and punctuation.  Here is my favorite example: people like naming a playlist "vibes" with many variants...

In [43]:
frames['PlaylistName'].query("PlaylistSimpleName == 'vibes'").head(20)

,PlaylistNameID,PlaylistName,PlaylistSimpleNameID,PlaylistSimpleName
904,905,#Vibes,16196,vibes
905,906,#vibes,16196,vibes
906,907,#vibes#,16196,vibes
1118,1119,((((((Vibes)))))),16196,vibes
1144,1145,((Vibes)),16196,vibes
1579,1580,*^viBes,16196,vibes
1959,1960,*VIBES,16196,vibes
1960,1961,*Vibes*,16196,vibes
1961,1962,*vibes*,16196,vibes
2332,2333,.Vibes,16196,vibes


For each playlist name, I computed a PlaylistSimpleName field by stripping out leading and trailing spaces (using the TRIM function), removing all punctuation, and moving all letters to lowercase.  This reduced the count of distinct playlist simple names to 17,131.  For each of these simple names, I then tabulated the frequency with which tracks appeared in all the playlists that shared that simple name.  The prediction for a new playlist starts with candidates that are the most popular tracks sharing that playlist name.  If necessary, this is padded by considering tracks with a high baseline frequency.  (This is a rare corner case but it does happen if a playlist name is obscure and only appears a small number of times, perhaps once.)  If a playlist name is not recognized, it is mapped to the empty string.  This is also where playlist names containing only punctuation or non-ascii characters are mapped.  Here is the SQL script that generates predictions using the playlist simple name model.

**Build the rank (by frequency) of each track keyed by PlaylistSimpleNameID**

`12_Import_TrackRankBySimpleName.sql`
```mysql
TRUNCATE TABLE dbo.TrackRankBySimpleName;
GO

-- CTE with the core group by query computing frequency of each 
-- (PlaylistSimpleNameID, TrackID) pair
WITH t1 AS (
SELECT
  pn.PlaylistSimpleNameID,
  pe.TrackID,
  COUNT(pe.TrackID) AS Frequency
FROM
  -- Start with all playlist entries
  dbo.PlaylistEntry AS pe
  -- The playlist for this entry
  INNER JOIN dbo.Playlist AS pl ON
    pl.PlaylistID = pe.PlaylistID
  -- Figure out if this playlist is train or test
  INNER JOIN dbo.TrainTestSplit AS tts ON
    tts.PlaylistID = pe.PlaylistID
  -- Get the PlaylistSimpleNameID from PlaylistName
  INNER JOIN dbo.PlaylistName AS pn ON
    pn.PlaylistName = pl.PlaylistName
WHERE
  -- Only training data!
  tts.TrainTestTypeID = 1
GROUP BY pn.PlaylistSimpleNameID, pe.TrackID
),
-- Join t1 against TrackRank to get BaselineFrequency and 
-- compute the rank of each track sharing the SimpleName
t2 AS(
SELECT
  t1.PlaylistSimpleNameID,
  row_number() OVER 
    (partition BY t1.PlaylistSimpleNameID
     ORDER BY t1.Frequency DESC, tr.Frequency DESC)
     AS TrackRank,
  t1.TrackID,
  t1.Frequency,
  tr.Frequency AS BaselineFrequency
FROM
  t1
  INNER JOIN dbo.TrackRank AS tr ON
    tr.TrackID = t1.TrackID
)
-- Insert into TrackRankBySimpleName from t2
INSERT INTO dbo.TrackRankBySimpleName
(PlaylistSimpleNameID, TrackRank, TrackID, Frequency, BaselineFrequency)
SELECT
  t2.PlaylistSimpleNameID,
  t2.TrackRank,
  t2.TrackID,
  t2.Frequency,
  t2.BaselineFrequency
FROM
  t2
WHERE
  -- Only compute ranks up to 1024 (need < 512 for prediction)
  t2.TrackRank <= 1024;
```

`32_MakeTable_Prediction_SimpleName.sql`

```mysql
CREATE TABLE dbo.Prediction_SimpleName(
PlaylistID INT NOT NULL,
Position SMALLINT NOT NULL,
TrackID INT NOT NULL,
Frequency INT NOT NULL,
BaselineFrequency INT NOT NULL,
TrackRank INT NOT NULL,
CONSTRAINT PK_Prediction_SimpleName_PlaylistID_Position
  PRIMARY KEY (PlaylistID, Position),
CONSTRAINT UNQ_Prediction_SimpleName_PlaylistID_TrackID
  UNIQUE (PlaylistID, TrackID),
CONSTRAINT FK_Prediction_SimpleName_PlaylistID
  FOREIGN KEY (PlaylistID) 
  REFERENCES dbo.Playlist(PlaylistID),
CONSTRAINT FK_Prediction_SimpleName_TrackID
  FOREIGN KEY (TrackID)
  REFERENCES dbo.Track(TrackID),  
);  
```

**Predict tracks on a playlist by taking the top tracks sharing its PlaylistSimpleName, eliminating duplicates.**

`02_Predict_SimpleName.sql`

```mysql
USE SpotifyDB;
GO

DECLARE @PlaylistCount AS INT = 1000000;
DECLARE @BatchSize AS INT = 1000;
DECLARE @i AS INT = 0;
DECLARE @p1 INT;
DECLARE @p2 INT;

WHILE (@i * @BatchSize) < @PlaylistCount
BEGIN

-- Range of playlists for this loop iteration
SET @p1 = (@i * @BatchSize + 1);
SET @p2 = ((@i+1) * @BatchSize);

-- Delete records in this block
DELETE pr
FROM dbo.Prediction_SimpleName AS pr
WHERE pr.PlaylistID BETWEEN @p1 AND @p2;

-- t1 is Playlists joined with candidate tracks sharing the simple name of the playlist
WITH t1 AS (
SELECT
  pl.PlaylistID,
  pl.NumTracks,
  trsn.TrackRank,
  trsn.TrackID,
  trsn.Frequency,
  trsn.BaselineFrequency
FROM
  -- Start with all playlists
  dbo.Playlist AS pl
  -- The simple name of the playlist
  INNER JOIN dbo.PlaylistName AS pn ON
    pn.PlaylistName = pl.PlaylistName
  -- The track frequencies for this playlist
  INNER JOIN dbo.TrackRankBySimpleName AS trsn ON
    trsn.PlaylistSimpleNameID = pn.PlaylistSimpleNameID 
    -- Only the 1024 top candidates
    AND trsn.TrackRank <= 1024
WHERE pl.PlaylistID BETWEEN @p1 AND @p2
),
-- Filter out tracks that are already there; compute the Position
t2 AS (
SELECT
  t1.PlaylistID,
  row_number() OVER
    (PARTITION BY t1.PlaylistID ORDER BY t1.Frequency DESC, t1.BaselineFrequency DESC)
      AS Position,
  t1.TrackID,
  t1.Frequency,
  t1.BaselineFrequency,
  t1.TrackRank
FROM
  t1  
-- Don't pick tracks that are already on the first (n-10) elements of the playlist!
WHERE NOT EXISTS 
  (SELECT pe.PlaylistID FROM dbo.PlaylistEntry AS pe 
   WHERE 
    pe.PlaylistID = t1.PlaylistID AND 
    pe.TrackID = t1.TrackID AND 
    pe.Position <= t1.NumTracks - 11)
)
-- Insert the first 512 positions into the Prediction table
INSERT INTO dbo.Prediction_SimpleName
(PlaylistID, Position, TrackID, Frequency, BaselineFrequency, TrackRank)
SELECT
  t2.PlaylistID,
  t2.Position,
  t2.TrackID,
  t2.Frequency,
  t2.BaselineFrequency,
  t2.TrackRank
FROM
  t2
WHERE
  t2.Position <= 512;

-- Status update; manual loop increment
PRINT CONCAT('Completed PlaylistID ', @i*@BatchSize+1, ' to ', (@i+1)*@BatchSize);
SET @i = @i+1;

END
```